In [32]:
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import sys
from PIL import Image, ImageDraw, ImageFont
import json
import cv2
import os
import time
import tensorflow as tf
from keras import layers, Input, models, optimizers
from keras.models import Model, Sequential
from keras.models import load_model

In [119]:
def style_n_pattern(img_dir, model_pattern, var_pattern, model_style, var_style):
    
    
    # 1) 사용할 model 설정
    model_pattern = load_model(model_pattern)    # pattern model
    model_style = load_model(model_style)        # style model
    
    
    # 2) 카카오 API 설정
    API_URL = 'https://kapi.kakao.com/v1/vision/product/detect'
    MYAPP_KEY = '890ebefd9a013403b1c6e290024eb8a3'
    #'89b7d96fc38f45fbf6ca52c2b13d2ad8'
    url = "https://kapi.kakao.com/v1/vision/product/detect"       
    headers = {'Authorization': 'KakaoAK {}'.format(MYAPP_KEY)}
    files = { 'file' : open(img_dir, 'rb')}
    response = requests.post(url, headers=headers, files=files)
    
    
    # 3) 카카오 API 저장 및 예외 상황 설정
    info = []
    result = response.json()
    
    if 'result' in result:                                                    # 카카오API가 이미지 인식 못하는 경우 stop
        fig_w, fig_h = result['result']['width'], result['result']['height']
    else:
        return ['NA','NA','NA']
    
    
    

    if 'objects' in result['result']:
        mylist = result['result']['objects']
        cnt = 0
        for objects in mylist:
            if objects['class'] in ['shirts','t-shirts','outer','pants','skirt']:
                cnt = cnt+1
        print(cnt)
        if cnt != 2:
            return ['NA','NA','NA']

        
#         if ob_list.count('shirts','t-shirts','outer','pants','skirt') > 2:            # 한 쌍 이상의 옷이 인식되는 경우 stop
#             return ['NA','NA','NA']   
        
#         if len(ob_list) < 2:                                                          # 상하의 하나라도 인식 안되는 경우 stop
#             return ['NA','NA','NA']   

        
        
        
    for each in result['result']['objects']:                        
            each = list(each.values())                                             # 순서 : x1, y1, x2, y2, class
            info.append(each)      
    
    
    
    
    
    
    # 4) 사진 상하의 구분 및 pattern 예측
    top = []
    bottom = []
    for _ in range(len(info)):
        img = Image.open(img_dir)
        category = info[_][4]

        if category in ('shirts','t-shirts','outer'):
            x = info[_][0]*fig_w - 5
            y = info[_][1]*fig_h - 5
            w = info[_][2]*fig_w + 5
            h = info[_][3]*fig_h + 5
            path = 'C:/Jupyter_Script/파이널프로젝트/img/cropped_img/top/'                  # 이미지 저장 코드
            name = path + category + str(time.time()) + '.jpg'                             # 이미지 저장 코드       
            img = img.crop((x,y,w,h))                               
            plt.imshow(img)
            img.show() 
            img.save(name)                                                                  # 이미지 저장 코드

            img = img.resize((150,150))
            data = np.array(img)
            data = data.astype('float')
            data = data/255
            data = data.reshape((1,150,150,3))
            top_pattern = var_pattern[model_pattern.predict_classes(data)[0]]
            top.append(top_pattern)

        elif category in ('pants','skirt'): 
            x = info[_][0]*fig_w - 5
            y = info[_][1]*fig_h - 5
            w = info[_][2]*fig_w + 5
            h = info[_][3]*fig_h + 5
            path = 'C:/Jupyter_Script/파이널프로젝트/img/cropped_img/bottom/'        # 이미지 저장 코드
            name = path + category + str(time.time()) + '.jpg'                       # 이미지 저장 코드  
            img = img.crop((x,y,w,h))                               
            plt.imshow(img)
            img.show()
            img.save(name)                                                           # 이미지 저장 코드

            img = img.resize((150,150))
            data = np.array(img)
            data = data.astype('float')
            data = data/255
            data = data.reshape((1,150,150,3))
            bottom_pattern = var_pattern[model_pattern.predict_classes(data)[0]]
            bottom.append(bottom_pattern)
    
    # 5) 사진 전체의 style 예측 
    img = Image.open(img_dir)
    img = img.resize((150,150))
    data = np.array(img)
    data = data.astype('float')
    data = data/255
    data = data.reshape((1,150,150,3))
    style = [var_style[model_style.predict_classes(data)[0]]]

    
    # 6)예측값이 없는 경우는 'NA' 값으로 표기
    if len(top) == 0:
        top = ['NA']
    if len(bottom) == 0:
        bottom = ['NA']
    if len(style) == 0:
        style = ['NA']
        
    return ( style + top + bottom )


In [52]:
def predict_multi(folder_dir, model_pattern, var_pattern, model_style, var_style):
    
    file_list = os.listdir(folder_dir)
    df = pd.DataFrame(index=range(0,0), columns=['img','style','top', 'bottom'])
    
    for _ in range(len(file_list)):
        img_dir = folder_dir + '/' + file_list[_]
        Image.open(img_dir)
        k = style_n_pattern(img_dir, model_pattern, var_pattern, model_style, var_style)
        total = [file_list[_]] + k
        df.loc[_] = total
        display(df)
    return df


In [ ]:
folder_dir = 'C:/Jupyter_Script/파이널프로젝트/img/test'
#img_dir = 'C:/Jupyter_Script/파이널프로젝트/img/데일리룩/데일리룩원본s1.jpg'
model_pattern = 'C:/Jupyter_Script/파이널프로젝트/model/pattern_no_camo_정확도76.h5'
var_pattern = ['check', 'dot', 'floral', 'graphic', 'leopard', 'none', 'stripe', 'tribal']   
model_style = 'C:/Jupyter_Script/파이널프로젝트/model/model_style_3개_90%_63%.h5'
var_style = ['casual', 'feminine', 'formal']


predict_multi(folder_dir, model_pattern, var_pattern, model_style, var_style)


### 왜 계속 result error??? 다른 폴더 이미지는 안그런데.. 
### 카카오 API에서 옷을 인식하지 못하는 경우!
### objects error 뜨는 것과 동일한 것
### key error : dict에서 key 값이 없어서 문제가 발생하는 것이기 때문에 해당 key 값이 존재하지 않는 경우는 NA 반환하도록 설정

3


,img,style,top,bottom
0,test (11).jpg,NA,NA,NA


2


,img,style,top,bottom
0,test (11).jpg,NA,NA,NA
1,test (12).jpg,casual,leopard,none
